In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#### 복습 
1. csv 폴더 안에 있는 drinks.csv 파일 로드 
2. 데이터프레임의 정보, 결측치가 존재하는가?
3. 컬럼의 이름을 변경 -> ['국가', '맥주소비량', '증류주소비량', '와인소비량', '소비알콜량', '대륙']
4. 대륙 컬럼에 결측치가 존재 -> 결측치를 'OT'로 대체
5. 새로운 파생변수(총 술소비량) 생성 -> 맥주소비량 + 증류주소비량 + 와인소비량
5. 새로운 파생변수(도수) 생성 -> 소비알콜량 / 총 술소비량 * 100
6. 도수 컬럼의 전체 평균보다 대륙별 도수 평균이 높은 대륙을 출력

In [ ]:
## csv 파일 로드 
drinks = pd.read_csv("../../csv/drinks.csv")
drinks.head(2)

In [ ]:
## 데이터프레임의 정보(info())를 출력 
drinks.info()

In [ ]:
## 결측치의 개수 
drinks.isna().sum()

In [ ]:
## 결측치가 존재하는 컬럼은 어디인가?
drinks.isna().any(axis=0)

In [ ]:
## 컬럼의 이름을 변경 
## case1 (rename())
drinks.rename(
    columns= {
        'country' : '국가', 
        'beer_servings' : '맥주소비량', 
        'spirit_servings' : '증류주소비량', 
        'wine_servings' : '와인소비량', 
        'total_litres_of_pure_alcohol' : '소비알콜량', 
        'continent' : '대륙'
    }
)

In [ ]:
## case2 ( DataFrame Class 안에 columns 변수의 데이터를 변경 )
drinks.columns = ['국가', '맥주소비량', '증류주소비량', '와인소비량', 
                  '소비알콜량', '대륙']

In [ ]:
# 대륙 컬럼의 결측치를 'OT' 대체
# fillna('OT')
drinks['대륙'].fillna('OT', inplace=True)

In [ ]:
drinks.isna().sum()

In [ ]:
drinks['대륙'].value_counts()

In [ ]:
## 파생변수 총 술소비량 -> 맥주소비량 + 증류주소비량 + 와인소비량
drinks['총 술소비량'] = \
    drinks['맥주소비량'] + drinks['증류주소비량'] + drinks['와인소비량']

In [ ]:
# 파생변수 도수 -> 소비알콜량 / 총 술소비량 * 100
drinks['도수'] = \
    (drinks['소비알콜량'] / drinks['총 술소비량']) * 100

In [ ]:
# 도수 컬럼의 결측치를 0으로 대체
drinks['도수'].fillna(0, inplace=True)

In [22]:
# 전체 도수의 평균 출력 
mean_data = drinks['도수'].mean()

In [25]:
# 대륙별 도수의 평균 출력 
group_data = drinks[ ['대륙', '도수'] ].groupby(['대륙']).mean()

In [ ]:
flag = group_data['도수'] > mean_data
flag

In [27]:
group_data.loc[flag]

,도수
대륙,
AF,4.974072
